## Limiting the number of cores that NumPyro can use

https://stackoverflow.com/questions/30791550/limit-number-of-threads-in-numpy?fbclid=IwAR3KpUvkrqQR5DR74zKoMFATxkFz_LxNOCoT2dCHNPEAEgohEHMDydSUyEU

In [1]:
#!pip install threadpoolctl

In [1]:
from threadpoolctl import threadpool_limits
from threadpoolctl import threadpool_info
from pprint import pprint
import numpy
pprint(threadpool_info())

#with threadpool_limits(limits=1, user_api='blas'):
  # single threaded numpy code...

[{'filepath': '/home/usuario/anaconda3/lib/python3.8/site-packages/numpy.libs/libopenblasp-r0-09e95953.3.13.so',
  'internal_api': 'openblas',
  'num_threads': 16,
  'prefix': 'libopenblas',
  'threading_layer': 'pthreads',
  'user_api': 'blas',
  'version': '0.3.13'}]


In [2]:
import os
#os.chdir("/content/")
# os.chdir("/Users/llaurabat/Dropbox/BGSE_work/LJRZH_graphs/graphical-regression-with-networks/numpyro")

os.chdir("/home/usuario/Documents/Barcelona_Yr1/GraphicalModels_NetworkData/Lauracode")

### Import modules, utils

In [3]:
import sys
#sys.path.append("./graphical-regression-with-networks/numpyro/functions")
sys.path.append("functions")

In [4]:
# save data to DRIVE, not to cloned repo
#data_save_path = './drive/MyDrive/graphical_reg_with_networks/stock_market_application/data/'
data_save_path = './simulations/final_models/data/'

sim_data_path = '/home/usuario/Documents/Barcelona_Yr1/GraphicalModels_NetworkData/LiLicode/Bayesian/GOLAZO_SS_SimData/data/'


In [5]:
import models
import my_utils

cpu
Is 64 precision enabled?: True
cpu
Is 64 precision enabled?: True


In [6]:
import importlib
importlib.reload(models)
importlib.reload(my_utils)

cpu
Is 64 precision enabled?: True
cpu
Is 64 precision enabled?: True


<module 'my_utils' from 'functions/my_utils.py'>

In [7]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from numpyro.util import enable_x64
import time
from datetime import timedelta

In [8]:
import jax
import numpyro
# numpyro.set_platform('gpu')
print(jax.lib.xla_bridge.get_backend().platform)

from jax import random, vmap
import jax.numpy as jnp

import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, log_likelihood
from jax.random import PRNGKey as Key
from numpyro.infer import init_to_feasible, init_to_value
from numpyro.diagnostics import print_summary
from numpyro.handlers import condition, substitute, block, seed
from optax import adam, exponential_decay
from numpyro.infer import SVI, Trace_ELBO
from numpyro.infer.autoguide import AutoDelta


cpu


In [9]:
enable_x64(use_x64=True)
print("Is 64 precision enabled?:", jax.config.jax_enable_x64)

Is 64 precision enabled?: True


In [17]:
p = 366
tril_idx = jnp.tril_indices(n=p, k=-1, m=p)
print(tril_idx)

(DeviceArray([  1,   2,   2, ..., 365, 365, 365], dtype=int64), DeviceArray([  0,   0,   1, ..., 362, 363, 364], dtype=int64))


## p = 10

In [10]:
n_sims = 10
p = 10
n = 2000
n_cut = 100
TP_thresh = 3

# Run full model with MCMC, reparametrisation and is_dense=False

In [11]:
# params
n_warmup = 2000
n_samples = 2000

mu_m=0.
mu_s=1.

#my_model = models.NetworkSS_repr
#my_model = models.golazo_ss_repr_etaRepr
#my_model = models.NetworkSS_repr_loglikRepr
my_model = models.NetworkSS_repr_etaRepr_loglikRepr
is_dense=False
verbose=False

estimates_print = ["w_slab", "mean_slab", "scale_slab"]

In [12]:
mu_fixed=jnp.zeros((p,))
scale_spike_fixed=0.003

In [13]:
# 85semidep params

eta0_0_m_85SEMI=0.
eta0_0_s_85SEMI=0.145
eta0_coefs_m_85SEMI=0.
eta0_coefs_s_85SEMI=0.145

eta1_0_m_85SEMI=-2.197
eta1_0_s_85SEMI=0.661
eta1_coefs_m_85SEMI=0.
eta1_coefs_s_85SEMI=0.661

eta2_0_m_85SEMI=-2.722
eta2_0_s_85SEMI=3.278
eta2_coefs_m_85SEMI=0.
eta2_coefs_s_85SEMI=3.278


In [14]:
# init strategy
#rho_init = jnp.diag(jnp.ones((p,)))
rho_tilde_init = jnp.zeros((int(p*(p-1)/2),))
u_init = jnp.ones((int(p*(p-1)/2),))*0.5

mu_init = jnp.zeros((p,))
sqrt_diag_init = jnp.ones((p,))

my_init_strategy_85SEMI = init_to_value(values={#'rho':rho_init,
                                                'rho_tilde':rho_tilde_init,
                                                'u':u_init,
                                                'mu':mu_init, 
                                                'sqrt_diag':sqrt_diag_init, 
                                                'tilde_eta0_0':0.0,
                                                'tilde_eta1_0':0.0,
                                                'tilde_eta2_0':0.0,
                                                'tilde_eta0_coefs':jnp.array([0.0]),
                                                'tilde_eta1_coefs':jnp.array([0.0]),
                                                'tilde_eta2_coefs':jnp.array([0.0]),})


#my_init_strategy = init_to_uniform()



In [15]:
diagnostics_all = {}
with threadpool_limits(limits=6, user_api='blas'):
    for s in range(n_sims):
        print('--------------------------------------------------------------------------------')
        print(f" Simulation number: {s} \n Dimensions: p = {p}, n = {n_cut} \n Run Network-SS with A-85SEMIDEP")
        with open(sim_data_path + f'sim{s}_p{p}_n{n}.sav', 'rb') as fr:
            sim_res = pickle.load(fr)

        A_list = [jnp.array(sim_res["A_scaled_semi_dep85"])]
        my_model_args = {"A_list":A_list, "eta0_0_m":eta0_0_m_85SEMI, "eta0_0_s":eta0_0_s_85SEMI, 
                 "eta0_coefs_m":eta0_coefs_m_85SEMI, "eta0_coefs_s":eta0_coefs_s_85SEMI,
                 "eta1_0_m":eta1_0_m_85SEMI, "eta1_0_s":eta1_0_s_85SEMI, 
                 "eta1_coefs_m":eta1_coefs_m_85SEMI, "eta1_coefs_s":eta1_coefs_s_85SEMI,
                 "eta2_0_m":eta2_0_m_85SEMI, "eta2_0_s":eta2_0_s_85SEMI, 
                 "eta2_coefs_m":eta2_coefs_m_85SEMI, "eta2_coefs_s":eta2_coefs_s_85SEMI,
                 "mu_m":mu_m, "mu_s":mu_s} 
    
        # select data 
        Y = jnp.array(sim_res['Y'])
        Y = Y[:n_cut,:]
        # summaries
        y_bar = Y.mean(axis=0) #p
        S_bar = Y.T@Y/n_cut - jnp.outer(y_bar, y_bar) #(p,p)
        my_model_args.update({"y_bar":y_bar, "S_bar":S_bar, "n":n_cut, "p":p})
        #my_model_args.update({"Y":Y, "n":n_cut, "p":p})
        theta_true = jnp.array(sim_res['theta_true'])
        tril_idx = jnp.tril_indices(n=p, k=-1, m=p)
        nonzero_true = (jnp.abs(theta_true[tril_idx]) != 0.)

        # set model
        fixed_params_dict = {"scale_spike":scale_spike_fixed, 
                         "mu":mu_fixed}
        blocked_params_list = ["scale_spike", "mu"]
        my_model_run = block(condition(my_model, fixed_params_dict), 
                         hide=blocked_params_list)


        nuts_kernel = NUTS(my_model_run, init_strategy=my_init_strategy_85SEMI, 
                       dense_mass=is_dense)
    
    
        # run model and time up
        start_time = time.time()
        mcmc = MCMC(nuts_kernel, num_warmup=n_warmup, num_samples=n_samples)
        #mcmc.run(rng_key = Key(s+44), Y=Y, **my_model_args,
        mcmc.run(rng_key = Key(s+44), **my_model_args,
            extra_fields=('potential_energy','accept_prob', 
                          'num_steps', 'adapt_state'))

        end_time = time.time()
        seconds_elapsed = end_time - start_time

        print(str(timedelta(seconds=seconds_elapsed)))

        # save samples
        res_all_samples = mcmc.get_samples()

        # record diagnostics
        params = ['eta0_0', 'eta0_coefs', 'eta1_0', 'eta1_coefs',
             'eta2_0', 'eta2_coefs', 'rho_tilde', 'rho_lt', 'sqrt_diag']

        diagnostics_dict = {'ESS':{}, 'r_hat':{}}

        for par in params:
            ESS = numpyro.diagnostics.summary(jnp.expand_dims(res_all_samples[par],
                                                          0))['Param:0']['n_eff']

            diagnostics_dict['ESS'][par] = ESS

            r_hat = numpyro.diagnostics.summary(jnp.expand_dims(res_all_samples[par],
                                                            0))['Param:0']['r_hat']

            diagnostics_dict['r_hat'][par] = r_hat

        diagnostics_dict.update({'potential_energy':mcmc.get_extra_fields()['potential_energy']})
        diagnostics_dict.update({'seconds_elapsed':seconds_elapsed})


        # save
        with open(data_save_path + f'diagnostics_ss_A85semi_{s}_p{p}_n{n_cut}.sav' , 'wb') as f:
            pickle.dump((diagnostics_dict), f)

        if verbose:
            # output indicators
            prob_slab_all = []
            for cs in range(n_samples):
                prob_slab = my_utils.get_prob_slab(rho_lt=res_all_samples['rho_lt'][cs], 
                                            mean_slab=res_all_samples['mean_slab'][cs], 
                                            scale_slab=res_all_samples['scale_slab'][cs], 
                                            scale_spike=fixed_params_dict['scale_spike'], 
                                            w_slab=res_all_samples['w_slab'][cs], 
                                            w_spike=(1-res_all_samples['w_slab'])[cs])
                prob_slab_all.append(prob_slab)
            prob_slab_est = (jnp.array(prob_slab_all)).mean(0)
            print('prob_slab_est', prob_slab_est.shape)

            nonzero_preds_5 = (prob_slab_est>0.5).astype(int)
            print(f'is_nonzero with thresh {0.5}', nonzero_preds_5)
            nonzero_preds_95 = (prob_slab_est>0.95).astype(int)
            print(f'is_nonzero with thresh {0.95}', nonzero_preds_95)


            TP_5 = jnp.where((nonzero_preds_5 == True)&(nonzero_true == True))[0].shape[0]
            FP_5 = jnp.where((nonzero_preds_5 == True)&(nonzero_true == False))[0].shape[0]
            FN_5 = jnp.where((nonzero_preds_5 == False)&(nonzero_true == True))[0].shape[0]
            TN_5 = jnp.where((nonzero_preds_5 == False)&(nonzero_true == False))[0].shape[0]

            TP_95 = jnp.where((nonzero_preds_95 == True)&(nonzero_true == True))[0].shape[0]
            FP_95 = jnp.where((nonzero_preds_95 == True)&(nonzero_true == False))[0].shape[0]
            FN_95 = jnp.where((nonzero_preds_95 == False)&(nonzero_true == True))[0].shape[0]
            TN_95 = jnp.where((nonzero_preds_95 == False)&(nonzero_true == False))[0].shape[0]

            theta = res_all_samples['theta'].mean(0)
            print('mse on estimated posterior mean for theta vs true theta: ', my_utils.get_MSE(theta, theta_true))
            print(" ")

            for k in estimates_print:
                est_mean = res_all_samples[f'{k}'].mean(0)
                print(f'estimated mean posterior for {k} is {est_mean}')

            print(" ") 
            print(f'Lower triangle of estimated theta is {theta[tril_idx]}')
            print(" ")
            print(f'Total for threshold 0.5:{p*(p-1)/2}, TP:{TP_5}, FP:{FP_5}, FN:{FN_5}, TN:{TN_5}')

            TPR = my_utils.get_TPR(TP=TP_5, FN=FN_5)
            FPR = my_utils.get_FPR(FP=FP_5, TN=TN_5)
            FNR = my_utils.get_FNR(FN=FN_5, TP=TP_5)
            print(f'TPR: {TPR}, FPR: {FPR}, FNR: {FNR}')
            try:
                FDiscR = my_utils.get_FDiscR(FP=FP_5, TP=TP_5)
                print(f'FDiscR: {FDiscR}')
            except:
                print('FDiscR: N/A, no positives')
            try:
                FNonDiscR = my_utils.get_FNonDiscR(TN=TN_5, FN=FN_5)
                print(f'FNonDiscR: {FNonDiscR}')
            except:
                print('FNonDiscR: N/A, no negatives')
        
        diagnostics_all[s] = diagnostics_dict
    with open(data_save_path + f'diagnostics_ss_A85semi_all_p{p}_n{n_cut}.sav' , 'wb') as f:
        pickle.dump((diagnostics_all), f)

--------------------------------------------------------------------------------
 Simulation number: 0 
 Dimensions: p = 10, n = 100 
 Run Network-SS with A-85SEMIDEP


  0%|          | 0/4000 [00:00<?, ?it/s]

0:00:27.532592
--------------------------------------------------------------------------------
 Simulation number: 1 
 Dimensions: p = 10, n = 100 
 Run Network-SS with A-85SEMIDEP


sample: 100%|██████████| 4000/4000 [00:37<00:00, 105.78it/s, 1023 steps of size 4.99e-03. acc. prob=0.96]


0:00:38.244036
--------------------------------------------------------------------------------
 Simulation number: 2 
 Dimensions: p = 10, n = 100 
 Run Network-SS with A-85SEMIDEP


...

0:00:28.156773
--------------------------------------------------------------------------------
 Simulation number: 8 
 Dimensions: p = 10, n = 100 
 Run Network-SS with A-85SEMIDEP


  0%|          | 0/4000 [00:00<?, ?it/s]

0:00:30.368923
--------------------------------------------------------------------------------
 Simulation number: 9 
 Dimensions: p = 10, n = 100 
 Run Network-SS with A-85SEMIDEP


sample: 100%|██████████| 4000/4000 [00:24<00:00, 162.47it/s, 127 steps of size 2.11e-02. acc. prob=0.60] 


0:00:25.064740


# Average diagnostics across simulations

In [17]:
# load diagnostics_all
with open(data_save_path + f'diagnostics_ss_A85semi_all_p{p}_n{n_cut}.sav', 'rb') as fr:
    diagnostics_all = pickle.load(fr)

In [18]:
params = ['eta0_0', 'eta0_coefs', 'eta1_0', 'eta1_coefs',
         'eta2_0', 'eta2_coefs', 'rho_lt']


In [19]:
avg_diagnostics = {}
for d in ['potential_energy', 'seconds_elapsed']:
    avg_d = []
    for s in range(n_sims):
        avg_d.append(diagnostics_all[s][d])

    avg_d = jnp.array(avg_d).mean(0)
    avg_diagnostics[d] = avg_d
    

In [20]:
avg_diagnostics.update({'ESS':{}, 'r_hat':{}})

In [21]:
for d in ['ESS', 'r_hat']:
    for par in params:
        avg_d_par = []
        for s in range(n_sims):
            avg_d_par.append(diagnostics_all[s][d][par])
        avg_d_par = jnp.array(avg_d_par).mean(0)  
        avg_diagnostics[d].update({par:avg_d_par})

In [22]:
avg_diagnostics

{'potential_energy': DeviceArray([1711.5835702 , 1713.5942495 , 1710.99282536, ...,
              1706.252708  , 1708.88649717, 1713.899879  ], dtype=float64),
 'seconds_elapsed': DeviceArray(28.98599925, dtype=float64),
 'ESS': {'eta0_0': DeviceArray(643.79097641, dtype=float64),
  'eta0_coefs': DeviceArray([527.16184097], dtype=float64),
  'eta1_0': DeviceArray(749.84371188, dtype=float64),
  'eta1_coefs': DeviceArray([487.85938066], dtype=float64),
  'eta2_0': DeviceArray(393.09305663, dtype=float64),
  'eta2_coefs': DeviceArray([332.77663487], dtype=float64),
  'rho_lt': DeviceArray([ 797.89707322,  958.76987544,  866.15431994, 1039.80789967,
               1036.26257315,  867.29874123, 1110.09581927,  836.79764716,
                644.52304977,  949.17736962, 1129.00811207, 1121.77043314,
                966.86320118, 1025.76169027, 1026.08458021, 1137.9181397 ,
               1158.27093674, 1106.51880435, 1066.03284721,  855.50382716,
                997.34671899, 1169.86552826, 

In [23]:
with open(data_save_path + f'diagnostics_ss_A85semi_avg_p{p}_n{n_cut}.sav' , 'wb') as f:
    pickle.dump((avg_diagnostics), f)
    

In [24]:
(avg_diagnostics['ESS']['eta0_0'] + avg_diagnostics['ESS']['eta0_coefs'] + 
 avg_diagnostics['ESS']['eta1_0'] + avg_diagnostics['ESS']['eta1_coefs'] + 
 avg_diagnostics['ESS']['eta2_0'] + avg_diagnostics['ESS']['eta2_coefs'])/6

DeviceArray([522.42093357], dtype=float64)

In [25]:
avg_diagnostics['ESS']['rho_lt'].mean()

DeviceArray(977.36267027, dtype=float64)

In [26]:
diagnostics_all

{0: {'ESS': {'eta0_0': 611.9942110194539,
   'eta0_coefs': array([711.74007071]),
   'eta1_0': 884.9290856763812,
   'eta1_coefs': array([796.50774556]),
   'eta2_0': 255.25463787242188,
   'eta2_coefs': array([344.30961243]),
   'rho_tilde': array([1438.68516507, 1094.4205754 , 1162.57557855, 1893.13423238,
          1731.56248849, 1267.07975245, 1557.09976447, 1411.73550365,
           646.17067197, 1104.33407267, 1444.18587445, 1580.03480093,
          1759.47539194, 1498.34733447, 1627.85393583, 1724.25038678,
          1740.87425153, 1582.63780516, 1131.90603917, 1976.97435902,
          1782.71635832, 1342.56441549, 1032.90802829, 1136.57448907,
          1189.4492062 , 2061.72382927,  965.85639182,  969.41084389,
          1522.67578577,  570.49751304,  260.48313128, 2270.04257458,
          1199.12243656, 1197.93472045, 1523.87008956,  556.62376658,
          1409.20585249, 1191.67056995, 1675.23859104, 1569.28885369,
          1426.12895384, 1349.31538802, 1864.3095229 , 1524.

## p = 50

In [27]:
n_sims = 10
p = 50
n = 2000
n_cut = 100
TP_thresh = 3

# Run full model with MCMC, reparametrisation and is_dense=False

In [28]:
# params
n_warmup = 2000
n_samples = 2000

mu_m=0.
mu_s=1.

#my_model = models.NetworkSS_repr
#my_model = models.NetworkSS_repr_etaRepr
my_model = models.NetworkSS_repr_etaRepr_loglikRepr
is_dense=False
verbose=False

estimates_print = ["w_slab", "mean_slab", "scale_slab"]

In [29]:
mu_fixed=jnp.zeros((p,))
scale_spike_fixed=0.003

In [30]:
# 85semidep params

##ARE THESE CORRECT 

eta0_0_m_85SEMI=0.
eta0_0_s_85SEMI=0.152
eta0_coefs_m_85SEMI=0.
eta0_coefs_s_85SEMI=0.152

eta1_0_m_85SEMI=-2.197
eta1_0_s_85SEMI=0.661
eta1_coefs_m_85SEMI=0.
eta1_coefs_s_85SEMI=0.661

eta2_0_m_85SEMI=-6.737
eta2_0_s_85SEMI=3.395
eta2_coefs_m_85SEMI=0.
eta2_coefs_s_85SEMI=3.395


In [31]:
# init strategy
#rho_init = jnp.diag(jnp.ones((p,)))
rho_tilde_init = jnp.zeros((int(p*(p-1)/2),))
u_init = jnp.ones((int(p*(p-1)/2),))*0.5

mu_init = jnp.zeros((p,))
sqrt_diag_init = jnp.ones((p,))

my_init_strategy_85SEMI = init_to_value(values={#'rho':rho_init,
                                                'rho_tilde':rho_tilde_init,
                                                'u':u_init,
                                                'mu':mu_init, 
                                                'sqrt_diag':sqrt_diag_init, 
                                                'tilde_eta0_0':0.0,
                                                'tilde_eta1_0':0.0,
                                                'tilde_eta2_0':0.0,
                                                'tilde_eta0_coefs':jnp.array([0.0]),
                                                'tilde_eta1_coefs':jnp.array([0.0]),
                                                'tilde_eta2_coefs':jnp.array([0.0]),})


#my_init_strategy = init_to_uniform()



In [32]:
diagnostics_all = {}
with threadpool_limits(limits=6, user_api='blas'):
    for s in range(n_sims):
        print('--------------------------------------------------------------------------------')
        print(f" Simulation number: {s} \n Dimensions: p = {p}, n = {n_cut} \n Run Network-SS with A-85SEMIDEP")
        with open(sim_data_path + f'sim{s}_p{p}_n{n}.sav', 'rb') as fr:
            sim_res = pickle.load(fr)

        A_list = [jnp.array(sim_res["A_scaled_semi_dep85"])]
        my_model_args = {"A_list":A_list, "eta0_0_m":eta0_0_m_85SEMI, "eta0_0_s":eta0_0_s_85SEMI, 
                 "eta0_coefs_m":eta0_coefs_m_85SEMI, "eta0_coefs_s":eta0_coefs_s_85SEMI,
                 "eta1_0_m":eta1_0_m_85SEMI, "eta1_0_s":eta1_0_s_85SEMI, 
                 "eta1_coefs_m":eta1_coefs_m_85SEMI, "eta1_coefs_s":eta1_coefs_s_85SEMI,
                 "eta2_0_m":eta2_0_m_85SEMI, "eta2_0_s":eta2_0_s_85SEMI, 
                 "eta2_coefs_m":eta2_coefs_m_85SEMI, "eta2_coefs_s":eta2_coefs_s_85SEMI,
                 "mu_m":mu_m, "mu_s":mu_s} 
    
        # select data 
        Y = jnp.array(sim_res['Y'])
        Y = Y[:n_cut,:]
        # summaries
        y_bar = Y.mean(axis=0) #p
        S_bar = Y.T@Y/n_cut - jnp.outer(y_bar, y_bar) #(p,p)
        my_model_args.update({"y_bar":y_bar, "S_bar":S_bar, "n":n_cut, "p":p,})
        theta_true = jnp.array(sim_res['theta_true'])
        tril_idx = jnp.tril_indices(n=p, k=-1, m=p)
        nonzero_true = (jnp.abs(theta_true[tril_idx]) != 0.)

        # set model
        fixed_params_dict = {"scale_spike":scale_spike_fixed, 
                         "mu":mu_fixed}
        blocked_params_list = ["scale_spike", "mu"]
        my_model_run = block(condition(my_model, fixed_params_dict), 
                         hide=blocked_params_list)


        nuts_kernel = NUTS(my_model_run, init_strategy=my_init_strategy_85SEMI, 
                       dense_mass=is_dense)
    
    
        # run model and time up
        start_time = time.time()
        mcmc = MCMC(nuts_kernel, num_warmup=n_warmup, num_samples=n_samples)
        #mcmc.run(rng_key = Key(s+44), Y=Y, **my_model_args,
        mcmc.run(rng_key = Key(s+44), **my_model_args,
            extra_fields=('potential_energy','accept_prob', 
                          'num_steps', 'adapt_state'))

        end_time = time.time()
        seconds_elapsed = end_time - start_time

        print(str(timedelta(seconds=seconds_elapsed)))

        # save samples
        res_all_samples = mcmc.get_samples()

        # record diagnostics
        params = ['eta0_0', 'eta0_coefs', 'eta1_0', 'eta1_coefs',
             'eta2_0', 'eta2_coefs', 'rho_tilde', 'rho_lt', 'sqrt_diag']

        diagnostics_dict = {'ESS':{}, 'r_hat':{}}

        for par in params:
            ESS = numpyro.diagnostics.summary(jnp.expand_dims(res_all_samples[par],
                                                          0))['Param:0']['n_eff']

            diagnostics_dict['ESS'][par] = ESS

            r_hat = numpyro.diagnostics.summary(jnp.expand_dims(res_all_samples[par],
                                                            0))['Param:0']['r_hat']

            diagnostics_dict['r_hat'][par] = r_hat

        diagnostics_dict.update({'potential_energy':mcmc.get_extra_fields()['potential_energy']})
        diagnostics_dict.update({'seconds_elapsed':seconds_elapsed})


        # save
        with open(data_save_path + f'diagnostics_ss_A85semi_{s}_p{p}_n{n_cut}.sav' , 'wb') as f:
            pickle.dump((diagnostics_dict), f)

        if verbose:
            # output indicators
            prob_slab_all = []
            for cs in range(n_samples):
                prob_slab = my_utils.get_prob_slab(rho_lt=res_all_samples['rho_lt'][cs], 
                                            mean_slab=res_all_samples['mean_slab'][cs], 
                                            scale_slab=res_all_samples['scale_slab'][cs], 
                                            scale_spike=fixed_params_dict['scale_spike'], 
                                            w_slab=res_all_samples['w_slab'][cs], 
                                            w_spike=(1-res_all_samples['w_slab'])[cs])
                prob_slab_all.append(prob_slab)
            prob_slab_est = (jnp.array(prob_slab_all)).mean(0)
            print('prob_slab_est', prob_slab_est.shape)

            nonzero_preds_5 = (prob_slab_est>0.5).astype(int)
            print(f'is_nonzero with thresh {0.5}', nonzero_preds_5)
            nonzero_preds_95 = (prob_slab_est>0.95).astype(int)
            print(f'is_nonzero with thresh {0.95}', nonzero_preds_95)


            TP_5 = jnp.where((nonzero_preds_5 == True)&(nonzero_true == True))[0].shape[0]
            FP_5 = jnp.where((nonzero_preds_5 == True)&(nonzero_true == False))[0].shape[0]
            FN_5 = jnp.where((nonzero_preds_5 == False)&(nonzero_true == True))[0].shape[0]
            TN_5 = jnp.where((nonzero_preds_5 == False)&(nonzero_true == False))[0].shape[0]

            TP_95 = jnp.where((nonzero_preds_95 == True)&(nonzero_true == True))[0].shape[0]
            FP_95 = jnp.where((nonzero_preds_95 == True)&(nonzero_true == False))[0].shape[0]
            FN_95 = jnp.where((nonzero_preds_95 == False)&(nonzero_true == True))[0].shape[0]
            TN_95 = jnp.where((nonzero_preds_95 == False)&(nonzero_true == False))[0].shape[0]

            theta = res_all_samples['theta'].mean(0)
            print('mse on estimated posterior mean for theta vs true theta: ', my_utils.get_MSE(theta, theta_true))
            print(" ")

            for k in estimates_print:
                est_mean = res_all_samples[f'{k}'].mean(0)
                print(f'estimated mean posterior for {k} is {est_mean}')

            print(" ") 
            print(f'Lower triangle of estimated theta is {theta[tril_idx]}')
            print(" ")
            print(f'Total for threshold 0.5:{p*(p-1)/2}, TP:{TP_5}, FP:{FP_5}, FN:{FN_5}, TN:{TN_5}')

            TPR = my_utils.get_TPR(TP=TP_5, FN=FN_5)
            FPR = my_utils.get_FPR(FP=FP_5, TN=TN_5)
            FNR = my_utils.get_FNR(FN=FN_5, TP=TP_5)
            print(f'TPR: {TPR}, FPR: {FPR}, FNR: {FNR}')
            try:
                FDiscR = my_utils.get_FDiscR(FP=FP_5, TP=TP_5)
                print(f'FDiscR: {FDiscR}')
            except:
                print('FDiscR: N/A, no positives')
            try:
                FNonDiscR = my_utils.get_FNonDiscR(TN=TN_5, FN=FN_5)
                print(f'FNonDiscR: {FNonDiscR}')
            except:
                print('FNonDiscR: N/A, no negatives')
        
        diagnostics_all[s] = diagnostics_dict
    with open(data_save_path + f'diagnostics_ss_A85semi_all_p{p}_n{n_cut}.sav' , 'wb') as f:
        pickle.dump((diagnostics_all), f)

--------------------------------------------------------------------------------
 Simulation number: 0 
 Dimensions: p = 50, n = 100 
 Run Network-SS with A-85SEMIDEP


sample: 100%|██████████| 4000/4000 [17:44<00:00,  3.76it/s, 511 steps of size 6.45e-03. acc. prob=0.73] 


0:17:48.745884


  0%|          | 0/4000 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
 Simulation number: 1 
 Dimensions: p = 50, n = 100 
 Run Network-SS with A-85SEMIDEP


...

0:16:26.833009


  0%|          | 0/4000 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
 Simulation number: 9 
 Dimensions: p = 50, n = 100 
 Run Network-SS with A-85SEMIDEP


sample: 100%|██████████| 4000/4000 [15:58<00:00,  4.17it/s, 511 steps of size 6.89e-03. acc. prob=0.91] 


0:15:59.022716


# Average diagnostics across simulations

In [33]:
# load diagnostics_all
with open(data_save_path + f'diagnostics_ss_A85semi_all_p{p}_n{n_cut}.sav', 'rb') as fr:
    diagnostics_all = pickle.load(fr)

In [34]:
params = ['eta0_0', 'eta0_coefs', 'eta1_0', 'eta1_coefs',
         'eta2_0', 'eta2_coefs', 'rho_lt']


In [35]:
avg_diagnostics = {}
for d in ['potential_energy', 'seconds_elapsed']:
    avg_d = []
    for s in range(n_sims):
        avg_d.append(diagnostics_all[s][d])

    avg_d = jnp.array(avg_d).mean(0)
    avg_diagnostics[d] = avg_d
    

In [36]:
avg_diagnostics.update({'ESS':{}, 'r_hat':{}})

In [37]:
for d in ['ESS', 'r_hat']:
    for par in params:
        avg_d_par = []
        for s in range(n_sims):
            avg_d_par.append(diagnostics_all[s][d][par])
        avg_d_par = jnp.array(avg_d_par).mean(0)  
        avg_diagnostics[d].update({par:avg_d_par})

In [38]:
avg_diagnostics

{'potential_energy': DeviceArray([12248.90968151, 12246.93928082, 12250.21024327, ...,
              12237.71417056, 12234.67451698, 12249.27917065],            dtype=float64),
 'seconds_elapsed': DeviceArray(1133.05708532, dtype=float64),
 'ESS': {'eta0_0': DeviceArray(220.31611614, dtype=float64),
  'eta0_coefs': DeviceArray([228.85831751], dtype=float64),
  'eta1_0': DeviceArray(340.0927537, dtype=float64),
  'eta1_coefs': DeviceArray([247.97172362], dtype=float64),
  'eta2_0': DeviceArray(358.013067, dtype=float64),
  'eta2_coefs': DeviceArray([361.13357108], dtype=float64),
  'rho_lt': DeviceArray([1044.71348048, 1665.29305856,  980.77922792, ...,
               1688.44589878, 1438.4277385 , 1492.50101771], dtype=float64)},
 'r_hat': {'eta0_0': DeviceArray(1.00576694, dtype=float64),
  'eta0_coefs': DeviceArray([1.00659675], dtype=float64),
  'eta1_0': DeviceArray(1.00395605, dtype=float64),
  'eta1_coefs': DeviceArray([1.00507422], dtype=float64),
  'eta2_0': DeviceArray(1.002690

In [39]:
with open(data_save_path + f'diagnostics_ss_A85semi_avg_p{p}_n{n_cut}.sav' , 'wb') as f:
    pickle.dump((avg_diagnostics), f)
    

In [40]:
(avg_diagnostics['ESS']['eta0_0'] + avg_diagnostics['ESS']['eta0_coefs'] + 
 avg_diagnostics['ESS']['eta1_0'] + avg_diagnostics['ESS']['eta1_coefs'] + 
 avg_diagnostics['ESS']['eta2_0'] + avg_diagnostics['ESS']['eta2_coefs'])/6

DeviceArray([292.73092484], dtype=float64)

In [41]:
avg_diagnostics['ESS']['rho_lt'].mean()

DeviceArray(1604.42435278, dtype=float64)

In [42]:
diagnostics_all

{0: {'ESS': {'eta0_0': 109.14812743177944,
   'eta0_coefs': array([123.2494586]),
   'eta1_0': 156.1054953755355,
   'eta1_coefs': array([128.41588473]),
   'eta2_0': 203.45995533250576,
   'eta2_coefs': array([219.91332668]),
   'rho_tilde': array([698.30596481, 926.83886509, 484.07871836, ..., 727.73718503,
          814.29397198, 415.12518655]),
   'rho_lt': array([ 573.54862947, 1156.41501987,  345.80685253, ..., 1112.04767088,
           967.17393765,  501.94198923]),
   'sqrt_diag': array([ 810.91632292,  720.49663778,  602.19883453,  706.6944548 ,
           526.83598101,  671.14013229,  921.0100424 ,  622.1399895 ,
           454.34626382,  624.06996262,  705.21636281,  726.8269935 ,
           610.47971079,  697.44061853,  718.34263078,  453.76069026,
           675.59224308,  867.13973872,  842.63163052,  717.44397926,
           685.81571457,  661.55106049,  575.65455826,  581.58710417,
           762.86762103,  645.51890946,  339.55652147,  559.07788429,
           922.7948